In [3]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

## Lets import our requirements
These are standard libraries used for our examples below

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
from torch.nn import Module
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Lets check we have gpu present, if so we will use gpu otherwise we use cpu

In [4]:
print(f"The device we are going to use (cpu/gpu):{device}")

The device we are going to use (cpu/gpu):cuda


# Sequence Reversing Problem

Here we are going to perform a very simple problem of reversing the order of a given sequence. Although this is a simple example, this tutorial will address all the steps in a Seq2Seq learning pipeline. This code can be easily extended to any Seq2Seq task like Translation, Question Answering etc.. (Take note of how the data needs to be fed to the pipeline).

## Dataset
The dataset can be easily created for this task, the code below will create the datasets and save it in a folder called ``./data``

The input sequence and the output sequence are created and are tab seperate as shown below

    a b c d e     e d c b a
    f a a e c     c e a a f

The dataset creation is controlled by few parameters,

``character_sample = "abcdef" `` this dicates the letters which we can use during the creation of the dataset.

``max_seq_length = 10`` this decides the maximum length a sequence can be in the dataset.

``num_line=100_000`` we control the number of datapoints in the dataset using this variable.


In [5]:
#this code will help us create a folder in colab
!mkdir "data"

In [6]:
import random

def generate_and_write_random_sequences_to_file(vocabulary, output_file, num_lines, max_seq_length,seed=7):
    """
    Generates a specified number of lines of random sequences from a vocabulary, respecting a maximum sequence length,
    reverses them, and writes to a text file.

    Args:
        vocabulary (str): A string containing the vocabulary words separated by commas.
        output_file (str): The path to the output text file.
        num_lines (int): The number of lines to generate and write.
        max_seq_length (int): The maximum sequence length.

    Returns:
        None
    """
    # Set a random seed
    np.random.seed(seed)
    # Split the vocabulary string into a list of words
    vocab_list = vocabulary.split(',')

    with open(output_file, 'w') as file:
        for _ in range(num_lines):
            # Randomly sample words from the vocabulary respecting the max sequence length
            seq_length = random.randint(1, max_seq_length)
            sampled_words = random.sample(vocab_list, min(seq_length, len(vocab_list)))

            # Generate the original sequence separated by spaces
            original_sequence = " ".join(sampled_words)

            # Reverse the sequence
            reversed_sequence = " ".join(sampled_words[::-1])

            # Write both sequences to the output file separated by tabs
            file.write(f"{original_sequence}\t{reversed_sequence}\n")


vocabulary = "a,b,c,d,e,f,g"
training_dataset = "./data/sequences_train.txt"
test_dataset = "./data/sequences_test.txt"

num_lines_train = 50_000
num_lines_test = 1000

max_seq_length = 10
generate_and_write_random_sequences_to_file(vocabulary, training_dataset, num_lines_train, max_seq_length,seed=7)
generate_and_write_random_sequences_to_file(vocabulary, test_dataset, num_lines_test, max_seq_length,seed=15)



## Data Processing

One of most crucial task in Seq2Seq is getting the data to a format which could be fed to the model. You MUST give equal importance as you do for modeling to this task. Basic Data Processing steps are,

*   Data Preprocessing : here we do transformation to the data if needed like lower casing the text, removing emojis, removing stop words etc..
*   Tokenization and Vocabulary creation : Discussed in the slides
*   Padding or Truncation : helps with batch processing

Since our data doesn't need any preprocessing, we shall start with tokenization and vocabulary creation.





### Tokenization and Vocabulary

This is a process of breaking the corpus into tokens(smallest unit for a paticular problem). The collection of all possible tokens is called the vocabulary. Vocabulary is like a lookup which will convert a token to a index (computers understand only numbers)

We'll need a unique index per character/subword/word to use as the inputs and targets of
the networks later. To keep track of all this we will use a helper class
called ``Vocabulary`` which has word → index (``word2index``) and index → word
(``index2word``) dictionaries, as well as a count of each word
``word2count`` which will be used to replace rare words later.




In [7]:
SOS_token = 0
EOS_token = 1

class Vocabulary:
    def __init__(self):
        # self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

The files are all in Unicode, to simplify we will turn Unicode
characters to ASCII, make everything lowercase, and trim most
punctuation.




In [8]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    return s.strip()

To read the data file we will split the file into lines, and then split
lines into pairs. The files are all English → Other Language, so if we
want to translate from Other Language → English I added the ``reverse``
flag to reverse the pairs.




In [9]:
def readData(path_data):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(path_data, encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    return pairs

In [10]:
def prepareData(path_data,verbose=False):
    pairs = readData(path_data)
    vocab = Vocabulary()
    for pair in pairs:
        vocab.addSentence(pair[0])
        vocab.addSentence(pair[1])
    if verbose:
      print("Read %s sentence pairs" % len(pairs))
      print("Counting words...")
      print("Counted words:")
      print(f"Vocabulary Size: {vocab.n_words}")
      print(f"Vocabulary :{vocab.index2word}")
    return pairs,vocab

pairs,vocab = prepareData(training_dataset, verbose=True)
print(random.choice(pairs))

Reading lines...
Read 50000 sentence pairs
Counting words...
Counted words:
Vocabulary Size: 9
Vocabulary :{0: 'SOS', 1: 'EOS', 2: 'c', 3: 'a', 4: 'g', 5: 'e', 6: 'b', 7: 'f', 8: 'd'}
['e b f a g c d', 'd c g a f b e']


### Preparing Training Data

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.

To prepare the data for training, first we need to map the tokens (in our case ``a, b, c, d, e, f`` to respective indexes, as shown above when we printed the vocabulary. This is taken care by ``indexesFromSentence`` method. Once the indices are generated, we need to make into pytorch tensors, this will be taken care by ``tensorFromSentence``. This will be done to both the ``source`` and ``target``.



In [11]:
MAX_LENGTH = 10
def indexesFromSentence(vocab, sentence):
    return [vocab.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(vocab, sentence):
    indexes = indexesFromSentence(vocab, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(vocab, pair[0])
    target_tensor = tensorFromSentence(vocab, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size,path_data):
    pairs,vocab = prepareData(path_data)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(vocab, inp)
        tgt_ids = indexesFromSentence(vocab, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return vocab, train_dataloader

Lets check the functionalities of these defined methods by sending some data

First lets look at how the token is mapped to index. Then let's make the returned indices into pytorch tensors.

In [12]:
sequence =  "a b c d"
token_mapped = indexesFromSentence(vocab,sequence)
print(f"{sequence}--> {token_mapped}")

mapped_indices_2_tensors = tensorFromSentence(vocab,sequence)
print(f"{token_mapped} --> {mapped_indices_2_tensors}")

a b c d--> [3, 6, 2, 8]
[3, 6, 2, 8] --> tensor([[3, 6, 2, 8, 1]], device='cuda:0')


**Note:** You can see an extra ``1`` added to pytorch tensor, this is the End of Sentence(EOS) token, which needs to be added at the end of all the sequence in the dataset, we will see the use of this when we look at the decoding part.



In order to utilize the gpu efficiently we need to process the data in batches, to create batches from our data we use the dataloader

In [13]:
#lets create the dataloader
batch_size = 2
vocab, train_dataloader = get_dataloader(batch_size,training_dataset)
#lets get the first batch
source, target = next(iter((train_dataloader)))
print("Source: ",source)
print("Tensor shape of Source: ",source.size())
print()
print("Target: ",target)
print("Tensor shape of Target: ",target.size())

Reading lines...
Source:  tensor([[4, 8, 6, 5, 3, 7, 2, 1, 0, 0],
        [3, 7, 1, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')
Tensor shape of Source:  torch.Size([2, 10])

Target:  tensor([[2, 7, 3, 5, 6, 8, 4, 1, 0, 0],
        [7, 3, 1, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')
Tensor shape of Target:  torch.Size([2, 10])


## The Seq2Seq Model

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A [Sequence to Sequence network](https://arxiv.org/abs/1409.3215)_, or
seq2seq network, or [Encoder Decoder
network](https://arxiv.org/pdf/1406.1078v3.pdf)_, is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the "meaning" of the input sequence into a single
vector — a single point in some N dimensional space of sentences.




### The Encoder

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.

.. figure:: /_static/img/seq-seq-images/encoder-network.png
   :alt:





In [14]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

### The Decoder

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




#### Simple Decoder

In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>``
token, and the first hidden state is the context vector (the encoder's
last hidden state).

.. figure:: /_static/img/seq-seq-images/decoder-network.png
   :alt:





In [15]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

I encourage you to train and observe the results of this model, but to
save space we'll be going straight for the gold and introducing the
Attention Mechanism.




#### Attention Decoder

If only the context vector is passed between the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to "focus" on a different part of
the encoder's outputs for every step of the decoder's own outputs. First
we calculate a set of *attention weights*. These will be multiplied by
the encoder output vectors to create a weighted combination. The result
(called ``attn_applied`` in the code) should contain information about
that specific part of the input sequence, and thus help the decoder
choose the right output words.

.. figure:: https://i.imgur.com/1152PYf.png
   :alt:

Calculating the attention weights is done with another feed-forward
layer ``attn``, using the decoder's input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.

.. figure:: /_static/img/seq-seq-images/attention-decoder-network.png
   :alt:


Bahdanau attention, also known as additive attention, is a commonly used
attention mechanism in sequence-to-sequence models, particularly in neural
machine translation tasks. It was introduced by Bahdanau et al. in their
paper titled [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/pdf/1409.0473.pdf)_.
This attention mechanism employs a learned alignment model to compute attention
scores between the encoder and decoder hidden states. It utilizes a feed-forward
neural network to calculate alignment scores.

However, there are alternative attention mechanisms available, such as Luong attention,
which computes attention scores by taking the dot product between the decoder hidden
state and the encoder hidden states. It does not involve the non-linear transformation
used in Bahdanau attention.

In this tutorial, we will be using Bahdanau attention. However, it would be a valuable
exercise to explore modifying the attention mechanism to use Luong attention.



In [16]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

<div class="alert alert-info"><h4>Note</h4><p>There are other forms of attention that work around the length
  limitation by using a relative position approach. Read about "local
  attention" in [Effective Approaches to Attention-based Neural Machine
  Translation](https://arxiv.org/abs/1508.04025)_.</p></div>

### Training the Model

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but [when the trained
network is exploited, it may exhibit
instability](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.378.4095&rep=rep1&type=pdf)_.

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation -
intuitively it has learned to represent the output grammar and can "pick
up" the meaning once the teacher tells it the first few words, but it
has not properly learned how to create the sentence from the translation
in the first place.

Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [17]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [18]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [19]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

### Plotting results

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.




In [20]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

## Evaluation

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [21]:
def evaluate(encoder, decoder, sentence, vocab):
    with torch.no_grad():
        input_tensor = tensorFromSentence(vocab, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(vocab.index2word[idx.item()])
    return decoded_words, decoder_attn

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [22]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('Input >', pair[0])
        print('Groud Truth = ', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], vocab)
        output_sentence = ' '.join(output_words)
        print('Generate < ', output_sentence)
        print('')

In [23]:
def evaluateWithAccuracy(encoder, decoder,pairs):
  correct_score = 0
  total = len(pairs)
  for source,target in pairs:
        output_words, _ = evaluate(encoder, decoder, source, vocab)
        output_sentence = ' '.join(output_words).replace("<EOS>","").strip()
        if target == output_sentence:
          correct_score+=1
  print(f"Accuracy of Model: {(correct_score/total)*100:.2f}%")

In [24]:
def evaluateSingleData(encoder, decoder,sentence):
    output_words, _ = evaluate(encoder, decoder, sentence, vocab)
    output_sentence = ' '.join(output_words)
    print('Input >', sentence)
    print('Generate < ', output_sentence)

## Training and Evaluating

With all these helper functions in place (it looks like extra work, but
it makes it easier to run multiple experiments) we can actually
initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small
dataset we can use relatively small networks of 256 hidden nodes and a
single GRU layer. After about 40 minutes on a MacBook CPU we'll get some
reasonable results.

.. Note::
   If you run this notebook you can train, interrupt the kernel,
   evaluate, and continue training later. Comment out the lines where the
   encoder and decoder are initialized and run ``trainIters`` again.




In [25]:
hidden_size = 128
batch_size = 32

vocab, train_dataloader = get_dataloader(batch_size,training_dataset)

encoder_without_attention = EncoderRNN(vocab.n_words, hidden_size).to(device)
decoder_without_attention = DecoderRNN(hidden_size, vocab.n_words).to(device)

train(train_dataloader, encoder_without_attention, decoder_without_attention, 10, print_every=5, plot_every=5)

Reading lines...
1m 22s (- 1m 22s) (5 50%) 0.0209
2m 42s (- 0m 0s) (10 100%) 0.0006


In [26]:
hidden_size = 128
batch_size = 32

vocab, train_dataloader = get_dataloader(batch_size,training_dataset)

encoder_with_attention = EncoderRNN(vocab.n_words, hidden_size).to(device)
decoder_with_attention = AttnDecoderRNN(hidden_size, vocab.n_words).to(device)

train(train_dataloader, encoder_with_attention, decoder_with_attention, 10, print_every=5, plot_every=5)

Reading lines...
2m 40s (- 2m 40s) (5 50%) 0.0154
5m 23s (- 0m 0s) (10 100%) 0.0000


In [27]:
test_data_pairs,_ = prepareData(test_dataset)

Reading lines...


Evaluating Mode without attention


In [28]:
encoder = encoder_without_attention
decoder = decoder_without_attention

encoder.eval()
decoder.eval()
evaluateWithAccuracy(encoder, decoder,test_data_pairs)

Accuracy of Model: 12.20%


Evaluating Mode with attention

In [29]:
encoder = encoder_with_attention
decoder = decoder_with_attention

encoder.eval()
decoder.eval()
evaluateWithAccuracy(encoder, decoder,test_data_pairs)

Accuracy of Model: 71.80%


Set dropout layers to ``eval`` mode



In [30]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

Input > g
Groud Truth =  g
Generate <  g g <EOS>

Input > e d
Groud Truth =  d e
Generate <  d e <EOS>

Input > a d g e b
Groud Truth =  b e g d a
Generate <  b e g d a <EOS>

Input > e
Groud Truth =  e
Generate <  e <EOS>

Input > b f a
Groud Truth =  a f b
Generate <  a f b <EOS>

Input > f
Groud Truth =  f
Generate <  f <EOS>

Input > f a b e c g d
Groud Truth =  d g c e b a f
Generate <  d g c e b a f <EOS>

Input > d f c g b e a
Groud Truth =  a e b g c f d
Generate <  a e b g c f d <EOS>

Input > a e f c b d g
Groud Truth =  g d b c f e a
Generate <  g d b c a e <EOS>

Input > d
Groud Truth =  d
Generate <  d d <EOS>



### Visualizing Attention

A useful property of the attention mechanism is its highly interpretable
outputs. Because it is used to weight specific encoder outputs of the
input sequence, we can imagine looking where the network is focused most
at each time step.

You could simply run ``plt.matshow(attentions)`` to see attention output
displayed as a matrix. For a better viewing experience we will do the
extra work of adding axes and labels:




In [31]:
def showAttention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence, vocab)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions[0, :len(output_words), :])

sequence = "a b c d e f a c d a f e"
evaluateAndShowAttention(sequence)

input = a b c d e f a c d a f e
output = e f a d c b a <EOS>


<ipython-input-31-d869f675164b>:8: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''] + input_sentence.split(' ') +
<ipython-input-31-d869f675164b>:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([''] + output_words)


In [32]:
evaluateSingleData(encoder_without_attention, decoder_without_attention, sequence)

Input > a b c d e f a c d a f e
Generate <  e f d c a <EOS>


# Credits
[Pytorch Tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

[Sequence To Sequence Learning With Tensorflow & Keras Tutorial Series](https://www.muratkarakaya.net/2022/11/seq2seq-learning-tutorial-series.html)


[Kaggle Tutorial on Sequence Reversing](https://www.kaggle.com/code/nadjetba/reverse-a-sequence-with-seq2seq/notebook)